## Install and Import

In [ ]:
!pip install wandb

In [2]:
!pip install spacy-transformers

In [42]:
import json
import random
import re

In [4]:
from spacy_transformers import Transformer, TransformerModel
from spacy_transformers.annotation_setters import null_annotation_setter
from spacy_transformers.span_getters import get_doc_spans
import spacy
from spacy import displacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Load the texts for inference

In [7]:
with open("/content/drive/MyDrive/cicero-nlp/2022_11_11/cleaned_texts/pure_texts.json", 'r') as f:
  pure_text_dict = json.load(f)

In [32]:
inference_sample = random.choice(list(pure_text_dict.values()))

In [33]:
inference_sample

"Representative Marvin W. Lucas - Biography - North Carolina General Assembly Employee Portal | Find Your Legislators | Contact Info Bill SEARCH: All Bill Text LOOKUP: Bill Members House Senate Audio House Chamber House Audio Archives Senate Chamber Finance Committee Room (544 LOB) Appropriations Committee Room (643 LOB) Committee Room 1027 LB Committee Room 1124 LB Committee Room 1228 LB Committee Room 415 LOB Committee Room 421 LOB Committee Room 423 LOB Press Conference Room Auditorium (LB) Audio Page Calendars House Calendar Senate Calendar Legislative Calendar Calendars Page Committees House Standing Committees Senate Standing Committees Non-Standing, Interim and Study Committees Committees Page Gov Ops Bills & Laws General Statutes General Statutes Table of Contents Session Laws Votes on Bills North Carolina Constitution Bills & Laws Page Divisions Legislative Library Fiscal Research Legislative Drafting Legislative Analysis General Assembly Police Divisions Page Legislative Publ

## Merge Model A

the inference process for Merge Model B and Address Model should be extract same

In [5]:
mergeModelA = spacy.load("/content/drive/MyDrive/cicero-nlp/modelCheckpoints/mergeModelA/20221207T103058/model-best")

In [52]:
result = mergeModelA(inference_sample)

In [53]:
# to visualized the inference results
displacy.render(result, style="ent", jupyter=True)

In [36]:
# get the extracted entities
predictions = result.ents
predictions

In [38]:
# get the labels for the entities
labels = [n.label_ for n in predictions]

In [39]:
# get the extracted entities with labels
predicitions_with_labels = list(zip(predictions, labels))
predicitions_with_labels

## Email

the inference for email is different and it is dependent on the results of Merge Model A

In [41]:
def extract_email_from_text(text):
  pattern = re.compile("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+")
  return re.findall(pattern, text)

In [55]:
# create the input for the email model, which should contains two parts
# (1) the extracted possible emails from the text by using Regex
# (2) the name entity predictions generated from Merge Model A
emails = extract_email_from_text(inference_sample)
email_input = " ".join(['Marvin', 'Lucas'] + emails)

In this case, we only have one email here.<br>
But in some cases, there might be multiple email found by using regex and not of them are the emails for the politican.

In [56]:
email_input

'Marvin Lucas Marvin.Lucas@ncleg.gov'

In [45]:
email_model = spacy.load("/content/drive/MyDrive/cicero-nlp/modelCheckpoints/email/20221208T211024/model-best")

In [57]:
result = email_model(email_input)

In [58]:
# to visualized the inference results
displacy.render(result, style="ent", jupyter=True)

In [59]:
# get the extracted entities
predictions = result.ents
predictions

(Marvin.Lucas@ncleg.gov,)

In [60]:
# get the labels for the entities
labels = [n.label_ for n in predictions]

In [61]:
# get the extracted entities with labels
predicitions_with_labels = list(zip(predictions, labels))
predicitions_with_labels

[(Marvin.Lucas@ncleg.gov, 'EMAIL')]